# 1.0


In [60]:
# import torch as th 
# import torch.nn as nn 
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import DataLoader, Dataset, random_split 

# import lightning as L
# from lightning import LightningModule, Trainer 


In [61]:
# import numpy as np
# embeddings_dict = {}
# with open(r"/Users/suyashsachdeva/Desktop/glove/glove.6B.50d.txt", 'r') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.asarray(values[1:], "float32")
#         embeddings_dict[word] = vector

In [62]:
# from nltk.stem import WordNetLemmatizer
 
# lemmatizer = WordNetLemmatizer()
# file = r'/Users/suyashsachdeva/Desktop/dialogs.txt'
# file = open(file, "r")

# def sentences(sent):
#         return [embeddings_dict[lemmatizer.lemmatize(''.join(e for e in word if e.isalnum()).lower())] for word in sent.split(" ")]

# # dataset = [[[embeddings_dict[lemmatizer.lemmatize(''.join(e for e in word if e.isalnum()).lower())] for word in sentence.split("\t")[0].split(" ")], [embeddings_dict[lemmatizer.lemmatize(''.join(e for e in word if e.isalnum()).lower())] for word in sentence.split("\t")[1].split(" ")]] for sentence in file.read().split("\n")]
# # dataset
# dataset = []
# for sent in file.read().replace("'ll", " will").replace("'ve", " have").replace("n't", " not").replace("101-98", "one zero one to nine eight").replace(",000", " thousand").replace("-", " ").split("\n"):
#     print(sent)
#     [inp, out] = sent.split("\t")
#     inp = sentences(inp.strip())
#     out = sentences(out.strip())
#     dataset.append([inp, out])
    


#  2.0

In [63]:
import torch as th 
import torch.nn as nn
import torch.nn.functional as F 
from torch.optim import Adam 
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import Variable

import lightning as L 
from lightning import Trainer, LightningModule

import torchtext as thtxt 
from torchtext.data import Field, BucketIterator, TabularDataset
from torchtext.vocab import FastText

import os 
import re 
import numpy as np

import spacy
nlp = spacy.load("en_core_web_lg")

In [69]:
def taka_tak_question(sentence):
    sentence = ["<sos>"] + [x for x in nlp(sentence)] + ["<eos>"]
    return ["<pad>"]*(25 - len(sentence)) + sentence

def taka_tak_answer(sentence):
    sentence = ["<sos>"] + [x for x in nlp(sentence)] + ["<eos>"]
    return  sentence + ["<pad>"]*(25 - len(sentence))


In [70]:

question = Field(sequential=True, use_vocab=False, include_lengths=True, tokenize=taka_tak_question, pad_first=True,  pad_token="<pad>", batch_first=True, fix_length=22)#, preprocessing=embedded)
answer   = Field(sequential=True, use_vocab=False, include_lengths=True,tokenize=taka_tak_answer, pad_first=False, pad_token="<pad>", batch_first=True, fix_length=22)#, preprocessing=embedded)

fields = {"question": ("q", question), "answer": ("a", answer)}
traindata, validdata = TabularDataset( path=r".../dialogs.tsv",
                                  format="tsv",
                                  fields=fields
                                        ).split()

In [ ]:
for word in traindata:
    print(nlp(word.a).vector, nlp(word.q).vector.shape)
    break

In [ ]:
length = [[len(tr.q), len(tr.a), len(va.q), len(va.q)]for tr, va in zip(traindata, validdata)]
max(np.array(length).flatten())

In [ ]:
type(embed)

In [75]:
traintensor, validtensor = BucketIterator.splits((traindata, validdata), batch_size=100, device="mps")

In [ ]:
for x in traintensor.q:
    print(x)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, inpsize, hidden, eos):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(inpsize, hidden)
        self.lstm = nn.LSTM(hidden, hidden)
        self.hid = hidden
        self.lstmcell = nn.LSTMCell(hidden, hidden)
        self.eos = eos
        
    def forward(self, txt):
        print(txt.size())
        h0 = Variable(th.zeros((1, 100, self.hid)))
        c0 = Variable(th.zeros((1, 100, self.hid)))
        embed = self.embedding(txt)
        output, (h, c) = self.lstm(embed, (h0, c0))
        h = h[-1]
        c = c[-1]
        x =output[-1]
        outputs = []
        while x!=eos:
            x, (h, c) = self.lstmcell(x, (h, c))
            outputs.append(x)
        return outputs

In [ ]:
class NewRNN(LightningModule):
    def __init__(self,data, model, lr):
        super(NewRNN, self).__init__()
        self.model = model 
        self.data = data 
        self.lr = lr 
        self.loss = nn.MSELoss()

    def forward(self, txt):
        return self.model(txt)
    
    def training_step(self, batch, bth_idx):
        question = batch.q 
        answer   = batch.a
        ypred = self(question) 
        loss = self.loss(ypred, answer)
        self.log("loss", loss)
        return loss
    
    def configure_optimizers(self):
        return Adam(self.model.parameters(), lr=self.lr)
    

    def train_dataloader(self):
        return self.data

In [ ]:
model = LSTMModel(300, 300, eos=eos)
learn = NewRNN(traintensor, model, lr=1e-4)
trainer = Trainer(min_epochs=3, max_epochs=10, accelerator="mps")
trainer.fit(learn)